![Podmanlogo](Pictures/podman-logo.png)

# Podman Networking

Podman uses the default network "podman" for all containers unless specified differently. If no other network is set all containers will be able to communicate with each other throught this network.

Nonetheless, there are scenarios in which you may want to create a separate netwokr for a set of containers. For example, if you have a three tier application with a frontend, backend and database. In this scenario you want the frontend to communicate with the backend but not with your database container directly as it will be using the backend for that. To achieve that you can create a "front" and a "back" network, then attach the frontend and the backend container to the "front" network, then attach the backend and the database containers to the "back" network. With this setup the backend container will be able to communicate with the frontend and the database as it's attached to both networks, but the fronend and database containers wont see each other. This is represented in the following picture:

![Multiplenetworks](Pictures/multiple-networks.png)

Lets see Podman networks in practice:

In [ ]:
!podman network create example-1

With the previous command we have created a Podman network called "example-1", the command has options to configure it (gatewayaddres, subnet mask, IPv4 or IPv6, etc). We can now use the following command to list all the networks we have:

In [ ]:
!podman network ls

As you can see we have our newly created "example-1" network and the default "podman" network available. We can remove our recently created network by running:

In [ ]:
!podman network rm example-1

Check if it was deleted:

In [ ]:
!podman network ls

You can also run the following command to remove all the unused networks at once:

In [ ]:
!podman network prune -f

Remember that Podman will attach a container to the default network when the network is not specified in the podman run command. This is important because the default network uses slirp4netns network mode while the ones you create manually will use the bridge network mode. If certain container is using slirp4netns network mode Podman wont be able to connect it to a bridged network and the command will fail.

Moreover, the default Podman network has the domain name system (DNS) resolution for other containers disabled. If you want to use DNS resolution between your containers create a dedicated network. When DNS is enabled the hostname for a container is the same as the container name.

> **Note**: There are other components, like firewall rules, that could be preventing the connection between containers in the same network.

Let's see a simple example of Podman networking in action. We are going to create two containers that will represent the frontend and backend containers of a simple application. First we will create both of them in separate networks and we will see how they're not able to connect.

Create the two podman networks:

In [ ]:
!podman network create frontend-network
!podman network create backend-network

Check all of them exist:

In [ ]:
!podman network ls

Next create the frontend container, backend and database container. We will be using nginx containers because it makes it easier to test connectivity by running a single curl command, in a real world scenario the containers would be different.

In [ ]:
!podman run -d --rm \
  --network frontend-network \
  --name frontend-container \
  docker.io/library/nginx:latest
!podman run -d --rm \
  --network backend-network \
  --name backend-container \
  docker.io/library/nginx:latest

Check the connection between containers by executing curl and trying to reach the other containers using their DNS names which is the same as their container name.
Try to reach the backend container from the frontend container:

In [ ]:
!podman exec frontend-container curl -s backend-container

No output, just as expected because they are sitting in different networks. We should get the same output if we try to reach the frontend from the backend container.

In [ ]:
!podman exec backend-container curl -s frontend-container

Again, we got nothing as expected. If we want them to be able to reach each other they need to be in the same network. The same container can sit in several networks at the same time so we will use the following command to connect the backend container to the frontend network:

In [ ]:
!podman network connect frontend-network backend-container

Now it's connected to the frontend network, so the frontend should be able to reach the backend.

In [ ]:
!podman exec frontend-container curl -s backend-container

It was able to communicate as expected. And the backend should be able to reach the frontend as well:

In [ ]:
!podman exec backend-container curl -s frontend-container

With this simple exercise we have demonstrated how simple it is to manage multiple networks and DNS resolution in a Podman environment.